In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from random import randint
import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch
from sklearn import *

Datasets:
https://scikit-learn.org/stable/datasets/toy_dataset.html
### Diabetes Dataset

In [2]:
X, Y = datasets.load_diabetes(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(442, 10) (442,)
(353, 10) (89, 10) (353,) (89,)


In [9]:
def my_func(config, reporter):  # add the reporter parameter
      # report the configuration
    clf = sklearn.neural_network.MLPRegressor(solver='lbfgs', alpha=config["alpha"],
                                            hidden_layer_sizes=(5,2),
                                            max_iter=90000,activation='relu',random_state=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    reporter(config, mean_loss = sklearn.metrics.mean_squared_error(y_test, y_pred))

In [ ]:
tune.register_trainable("my_func", my_func)
ray.shutdown()
ray.init()

res = tune.run_experiments({
    "my_experiment": {
        "run": "my_func",
        "stop": { "mean_accuracy": 4000 },
        "config": {
            "alpha": tune.grid_search([0.1, 0.2]),
            "hidden_layer_sizes": tune.grid_search([(5,2), (10,2)]),
        }
    }
})

In [10]:
algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
analysis = tune.run(
        my_func,
        name="my_exp",
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=10,
        config={
            "steps": 100,
            "alpha": tune.uniform(0.0001, 10),
        },
    )

print("Best hyperparameters found were: ", analysis.best_config)

2022-04-18 06:47:03,047	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


Trial name,status,loc,alpha
my_func_972f771d,RUNNING,,3.74546


Result for my_func_975fb4e1:
  _metric:
    alpha: 7.31996621871987
    steps: 100
  date: 2022-04-18_06-47-06
  done: false
  experiment_id: 8e2dc2330ef5480082b73bf4b879b34f
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 2636.446960233829
  neg_mean_loss: -2636.446960233829
  node_ip: 192.168.1.106
  pid: 5252
  time_since_restore: 0.14760565757751465
  time_this_iter_s: 0.14760565757751465
  time_total_s: 0.14760565757751465
  timestamp: 1650257226
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 975fb4e1
  
Result for my_func_975fb4e1:
  _metric:
    alpha: 7.31996621871987
    steps: 100
  date: 2022-04-18_06-47-06
  done: true
  experiment_id: 8e2dc2330ef5480082b73bf4b879b34f
  experiment_tag: 3_alpha=7.32,steps=100
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 2636.446960233829
  neg_mean_loss: -2636.446960233829
  node_ip: 192.168.1.106
  pid: 5252
  time_since_restore: 0.14760565757751465
  time_this_iter_s: 0.14760565757751465
 

Trial name,status,loc,alpha,loss,iter,total time (s),neg_mean_loss
my_func_9778aa16,RUNNING,,5.98662,,,,
my_func_9957e182,RUNNING,,1.56027,,,,
my_func_995dd0de,RUNNING,,1.56003,,,,
my_func_996ac07e,PENDING,,0.58093,,,,
my_func_972f771d,TERMINATED,,3.74546,2639.76,1,0.273269,-2639.76
my_func_97497d00,TERMINATED,,9.50715,2634.48,1,0.226393,-2634.48
my_func_975fb4e1,TERMINATED,,7.31997,2636.45,1,0.147606,-2636.45


Result for my_func_9957e182:
  _metric:
    alpha: 1.560270802560321
    steps: 100
  date: 2022-04-18_06-47-17
  done: true
  experiment_id: 4288d0a5bb1f48469a7f9cd11025416c
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 2640.521885238005
  neg_mean_loss: -2640.521885238005
  node_ip: 192.168.1.106
  pid: 7632
  time_since_restore: 0.3640265464782715
  time_this_iter_s: 0.3640265464782715
  time_total_s: 0.3640265464782715
  timestamp: 1650257237
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9957e182
  


Trial name,status,loc,alpha,loss,iter,total time (s),neg_mean_loss
my_func_9778aa16,RUNNING,,5.98662,,,,
my_func_995dd0de,RUNNING,,1.56003,,,,
my_func_996ac07e,RUNNING,,0.58093,,,,
my_func_972f771d,TERMINATED,,3.74546,2639.76,1,0.273269,-2639.76
my_func_97497d00,TERMINATED,,9.50715,2634.48,1,0.226393,-2634.48
my_func_975fb4e1,TERMINATED,,7.31997,2636.45,1,0.147606,-2636.45
my_func_9957e182,TERMINATED,,1.56027,2640.52,1,0.364027,-2640.52


Result for my_func_9778aa16:
  _metric:
    alpha: 5.986624976121946
    steps: 100
  date: 2022-04-18_06-47-17
  done: true
  experiment_id: e494a41b920944e395b16ab4914a5160
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 2637.1706114940666
  neg_mean_loss: -2637.1706114940666
  node_ip: 192.168.1.106
  pid: 16900
  time_since_restore: 0.45478129386901855
  time_this_iter_s: 0.45478129386901855
  time_total_s: 0.45478129386901855
  timestamp: 1650257237
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9778aa16
  
Result for my_func_996ac07e:
  _metric:
    alpha: 0.5809303133207778
    steps: 100
  date: 2022-04-18_06-47-17
  done: true
  experiment_id: 9ef40748e38349228d908f77d27c3b48
  hostname: Ghallab
  iterations_since_restore: 1
  mean_loss: 2641.618386699426
  neg_mean_loss: -2641.618386699426
  node_ip: 192.168.1.106
  pid: 6020
  time_since_restore: 0.3321101665496826
  time_this_iter_s: 0.3321101665496826
  time_total_s: 0.3321101665496826
  ti

Trial name,status,loc,alpha,loss,iter,total time (s),neg_mean_loss
my_func_972f771d,TERMINATED,,3.74546,2639.76,1,0.273269,-2639.76
my_func_97497d00,TERMINATED,,9.50715,2634.48,1,0.226393,-2634.48
my_func_975fb4e1,TERMINATED,,7.31997,2636.45,1,0.147606,-2636.45
my_func_9778aa16,TERMINATED,,5.98662,2637.17,1,0.454781,-2637.17
my_func_9957e182,TERMINATED,,1.56027,2640.52,1,0.364027,-2640.52
my_func_995dd0de,TERMINATED,,1.56003,2515.66,1,0.593413,-2515.66
my_func_996ac07e,TERMINATED,,0.58093,2641.62,1,0.33211,-2641.62
my_func_9fa10b09,TERMINATED,,8.66177,2635.21,1,0.233377,-2635.21
my_func_9faa7a6c,TERMINATED,,6.01119,2637.39,1,0.373003,-2637.39
my_func_9fc56a13,TERMINATED,,7.08075,2636.41,1,0.475727,-2636.41


2022-04-18 06:47:22,542	INFO tune.py:561 -- Total run time: 19.51 seconds (18.00 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 100, 'alpha': 1.5600296039099928}
